In [12]:
# Handle table-like data and matrices
import numpy as np
import pandas as pd
from sympy import *
import scipy
from scipy.stats import fisher_exact,uniform
# Helpers
import os
import sys
sys.path.insert(0,'../')
from scipy.special import digamma,betaln
import time

# Prediction
from classifiers import MAP_estimator,cal_p_value,TCRs_selection,LOOCV_MAP

from numpy import random
from sklearn.metrics import accuracy_score, roc_curve, roc_auc_score,confusion_matrix,log_loss
from sklearn.model_selection import LeaveOneOut,KFold,StratifiedKFold

# Visualisation
import matplotlib.pyplot as plt
import seaborn as sns
plt.style.use('ggplot')
from mpl_toolkits.mplot3d import Axes3D
%matplotlib inline

# import plotly
# plotly.tools.set_credentials_file(username='tracyqin326', api_key='EICCf5vuIzI5hVfA4gYC')
# import plotly.plotly as py
# import plotly.graph_objs as go

# Ignore warnings
import warnings
warnings.filterwarnings('ignore')

## Data

In [2]:
train_origin = pd.read_csv('../data/'+'train.csv')
count_df = pd.read_pickle('../data/'+'count_df.pkl')
freq_df = pd.read_pickle('../data/freq_df.pkl')

In [3]:
count_df.head(10)

,sample_name,phenotype_status,"(CAAGGSSYEQYF, TCRBV07, TCRBV07-08, 01, TCRBJ02, TCRBJ02-07, 01)","(CACLPGQTSYEQYF, TCRBV30, TCRBV30-01, 01, TCRBJ02, TCRBJ02-07, 01)","(CACRGEGGNTIYF, TCRBV30, TCRBV30-01, 01, TCRBJ01, TCRBJ01-03, 01)","(CAFGQEGQPQHF, TCRBV30, TCRBV30-01, 01, TCRBJ01, TCRBJ01-05, 01)","(CAGETAEAFF, TCRBV06, TCRBV06-01, 01, TCRBJ01, TCRBJ01-01, 01)","(CAGGMNTEAFF, TCRBV30, TCRBV30-01, 01, TCRBJ01, TCRBJ01-01, 01)","(CAGGNTEAFF, TCRBV02, TCRBV02-01, 01, TCRBJ01, TCRBJ01-01, 01)","(CAGGRAGGTDTQYF, TCRBV07, TCRBV07-09, null, TCRBJ02, TCRBJ02-03, 01)",...,"(CSVWTGSSYEQYF, TCRBV29, TCRBV29-01, 01, TCRBJ02, TCRBJ02-07, 01)","(CSVYLPGQGNGGYGYTF, TCRBV29, TCRBV29-01, 01, TCRBJ01, TCRBJ01-02, 01)","(CSVYMNTEAFF, TCRBV29, TCRBV29-01, 01, TCRBJ01, TCRBJ01-01, 01)","(CSVYNNEQFF, TCRBV29, TCRBV29-01, 01, TCRBJ02, TCRBJ02-01, 01)","(CSVYRDYEQYF, TCRBV29, TCRBV29-01, 01, TCRBJ02, TCRBJ02-07, 01)","(CSVYTGELFF, TCRBV29, TCRBV29-01, 01, TCRBJ02, TCRBJ02-02, 01)","(CTGLHETQYF, TCRBV27, TCRBV27-01, 01, TCRBJ02, TCRBJ02-05, 01)","(CVTRDFYEQYF, TCRBV02, TCRBV02-01, 01, TCRBJ02, TCRBJ02-07, 01)","(RASSPFPTNQAASSNQPQHF, TCRBV07, TCRBV07-03, 03, TCRBJ01, TCRBJ01-05, 01)","(RASSSTSGGPSWNDEQFF, TCRBV07, TCRBV07-03, 03, TCRBJ02, TCRBJ02-01, 01)"
0,RA47,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,HC9,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,1,2
2,RA29,1,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,RA8,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,RA63,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,RA33,1,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,HC17,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
7,HC3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,RA17,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,RA46,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [4]:
freq_df.head(10)

,sample_name,total_TCRs,phenotype_status,"(CAAGGSSYEQYF, TCRBV07, TCRBV07-08, 01, TCRBJ02, TCRBJ02-07, 01)","(CACLPGQTSYEQYF, TCRBV30, TCRBV30-01, 01, TCRBJ02, TCRBJ02-07, 01)","(CACRGEGGNTIYF, TCRBV30, TCRBV30-01, 01, TCRBJ01, TCRBJ01-03, 01)","(CAFGQEGQPQHF, TCRBV30, TCRBV30-01, 01, TCRBJ01, TCRBJ01-05, 01)","(CAGETAEAFF, TCRBV06, TCRBV06-01, 01, TCRBJ01, TCRBJ01-01, 01)","(CAGGMNTEAFF, TCRBV30, TCRBV30-01, 01, TCRBJ01, TCRBJ01-01, 01)","(CAGGNTEAFF, TCRBV02, TCRBV02-01, 01, TCRBJ01, TCRBJ01-01, 01)",...,"(CSVWTGSSYEQYF, TCRBV29, TCRBV29-01, 01, TCRBJ02, TCRBJ02-07, 01)","(CSVYLPGQGNGGYGYTF, TCRBV29, TCRBV29-01, 01, TCRBJ01, TCRBJ01-02, 01)","(CSVYMNTEAFF, TCRBV29, TCRBV29-01, 01, TCRBJ01, TCRBJ01-01, 01)","(CSVYNNEQFF, TCRBV29, TCRBV29-01, 01, TCRBJ02, TCRBJ02-01, 01)","(CSVYRDYEQYF, TCRBV29, TCRBV29-01, 01, TCRBJ02, TCRBJ02-07, 01)","(CSVYTGELFF, TCRBV29, TCRBV29-01, 01, TCRBJ02, TCRBJ02-02, 01)","(CTGLHETQYF, TCRBV27, TCRBV27-01, 01, TCRBJ02, TCRBJ02-05, 01)","(CVTRDFYEQYF, TCRBV02, TCRBV02-01, 01, TCRBJ02, TCRBJ02-07, 01)","(RASSPFPTNQAASSNQPQHF, TCRBV07, TCRBV07-03, 03, TCRBJ01, TCRBJ01-05, 01)","(RASSSTSGGPSWNDEQFF, TCRBV07, TCRBV07-03, 03, TCRBJ02, TCRBJ02-01, 01)"
0,RA47,16724,1,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000
1,HC9,15101,0,0.000000,0.000066,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000066,0.000066,0.000132
2,RA29,22765,1,0.000000,0.000000,0.000044,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000
3,RA8,6333,1,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000
4,RA63,4849,1,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000
5,RA33,8855,1,0.000113,0.000000,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000
6,HC17,30233,0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000033,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000
7,HC3,13774,0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000
8,RA17,2314,1,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000
9,RA46,19708,1,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000


In [14]:
train_origin.head()

,sample_name,total_TCRs,unique_TCRs,0.1,0.2,0.3,0.4,0.5,0.6,0.7,phenotype_status
0,RA47,16724,16495,0,5,20,44,139,616,640,1
1,HC9,15101,14907,0,1,1,2,4,6,15,0
2,RA29,22765,22462,1,9,34,81,223,882,929,1
3,RA8,6333,6143,0,2,6,13,43,164,170,1
4,RA63,4849,4735,2,3,6,20,61,226,230,1


In [5]:

TCRs = count_df.drop(['sample_name','phenotype_status'],axis=1).columns.values # a list of TCRs candidates (TCRs on the whole dataset)

# Init lists
sample_name = []
y_true = []
y_pred = []
y_proba = [] # used to store positive-class posterior probability using posterior_c1 method
y_proba_v2 = [] # used to test the method predict_proba

kf = LeaveOneOut()
for train_index,test_index in kf.split(train_origin): # for each cv round

    train = train_origin.copy(deep=True) # a copy of the original training data
    train_cv, test_cv = train.iloc[train_index], train.iloc[test_index] # get training samples and one testing sample

    # Select a list of associated TCRs based on count df of training samples and threshold
    count_train = freq_df[count_df['sample_name'].isin(train_cv['sample_name'])] # count df of training samples
    count_test = freq_df[count_df['sample_name'].isin(test_cv['sample_name'])] # count df of the testing sample
    break

In [6]:
neg_df = count_train[count_train['phenotype_status']==0]
pos_df = count_train[count_train['phenotype_status']==1]

## Test T-test

In [13]:
from scipy.stats import ttest_ind

In [8]:
a = pos_df[TCRs[0]]
b = neg_df[TCRs[0]]

In [9]:
print(np.mean(a),np.mean(b))

1.764539808018069e-06 3.4020548411240386e-06


In [10]:
ttest_ind(a,b,equal_var=False)

Ttest_indResult(statistic=-0.42727767770198793, pvalue=0.6722339872054206)

In [11]:
for i in range(20):
    pos = pos_df[TCRs[i]]
    neg = neg_df[TCRs[i]]
    print('pos_mean:',np.mean(pos),'neg_mean:',np.mean(neg))
    if np.mean(pos)!=0 or np.mean(neg)!=0:
        t_stat, pvalue = ttest_ind(pos,neg,equal_var=False)
        if t_stat > 0:
            pvalue = pvalue/2
        else:
            pvalue = 1-(pvalue/2)
        print('real pvalue:',pvalue)
        print()
    else:
        print('No such TCR exists\n')

pos_mean: 1.764539808018069e-06 neg_mean: 3.4020548411240386e-06
real pvalue: 0.6638830063972897

pos_mean: 0.0 neg_mean: 4.84280789074984e-06
real pvalue: 0.9043303768731259

pos_mean: 1.4524046674176503e-06 neg_mean: 0.0
real pvalue: 0.07978223696490543

pos_mean: 1.1805002595040178e-06 neg_mean: 0.0
real pvalue: 0.08863653077761809

pos_mean: 4.638156256002803e-06 neg_mean: 0.0
real pvalue: 0.08794061621290888

pos_mean: 2.1819654080796955e-06 neg_mean: 0.0
real pvalue: 0.06309799282375535

pos_mean: 1.6225125191802168e-06 neg_mean: 0.0
real pvalue: 0.10320975559574709

pos_mean: 0.0 neg_mean: 0.0
No such TCR exists

pos_mean: 2.2551319894864414e-06 neg_mean: 0.0
real pvalue: 0.09005400983229525

pos_mean: 7.66044026082267e-07 neg_mean: 5.958055290753098e-06
real pvalue: 0.801072855252145

pos_mean: 0.0 neg_mean: 0.0
No such TCR exists

pos_mean: 6.706296407571141e-07 neg_mean: 2.462690242821258e-06
real pvalue: 0.7549966063548796

pos_mean: 0.0 neg_mean: 3.891372473307825e-06
real 

In [24]:
def independent_ttest(data1, data2, alpha):
    # calculate means
    mean1, mean2 = np.mean(data1), np.mean(data2)
    # calculate standard errors
    se1, se2 = scipy.stats.sem(data1), scipy.stats.sem(data2)
    # standard error on the difference between the samples
    sed = np.sqrt(se1**2.0 + se2**2.0)
    # calculate the t statistic
    t_stat = (mean1 - mean2) / sed
    # degrees of freedom
    df = len(data1) + len(data2) - 2
    # calculate the critical value
    cv = scipy.stats.t.ppf(1.0 - alpha, df)
    # calculate the p-value
    p = (1.0 - scipy.stats.t.cdf(abs(t_stat), df)) * 2.0
    # return everything
    return t_stat, df, cv, p

### Customed Function

In [43]:
TCRs_pvalue = dict() # init dict
for tcr in TCRs: # for each TCR in the list of TCR candidates

    pos = pos_df[tcr]
    neg = neg_df[tcr]

    if np.mean(pos)!=0 or np.mean(neg)!=0: # if does not occur in any classes, not to add to the dict 
        TCRs_pvalue[tcr] = [np.mean(pos),np.mean(neg)] # add the TCR incidence to dict
        p_value = t_test(pos,neg)
        TCRs_pvalue[tcr].append(p_value) # store p-value

p_value_df = pd.DataFrame.from_dict(TCRs_pvalue,orient='index') # generate df from TCR incidence dict
p_value_df.columns = ['mean(pos)','mean(neg)','p_value'] # rename df


In [47]:
p_value_df.sort_values(by='p_value')

,mean(pos),mean(neg),p_value
"(CASSYSGGNQPQHF, TCRBV06, TCRBV06-05, 01, TCRBJ01, TCRBJ01-05, 01)",7.622900e-06,0.000000,0.003587
"(CASSIGTEAFF, TCRBV19, TCRBV19-01, null, TCRBJ01, TCRBJ01-01, 01)",1.240914e-05,0.000000,0.003954
"(CASSPGTGAYEQYF, TCRBV06, TCRBV06-05, 01, TCRBJ02, TCRBJ02-07, 01)",1.018503e-05,0.000000,0.004204
"(CASSLGYEQYF, TCRBV05, TCRBV05-01, 01, TCRBJ02, TCRBJ02-07, 01)",3.650590e-05,0.000004,0.004303
"(CASSQGLNTEAFF, TCRBV04, TCRBV04-01, 01, TCRBJ01, TCRBJ01-01, 01)",7.163364e-06,0.000000,0.004631
"(CASSYQGGNYGYTF, TCRBV06, TCRBV06-05, 01, TCRBJ01, TCRBJ01-02, 01)",1.984703e-05,0.000002,0.005290
"(CASSVGGGAYNEQFF, TCRBV09, TCRBV09-01, null, TCRBJ02, TCRBJ02-01, 01)",6.365540e-06,0.000000,0.005474
"(CASSVGYNEQFF, TCRBV09, TCRBV09-01, null, TCRBJ02, TCRBJ02-01, 01)",1.074074e-05,0.000000,0.005905
"(CASSQDSYEQYF, TCRBV04, TCRBV04-02, 01, TCRBJ02, TCRBJ02-07, 01)",2.835049e-05,0.000000,0.005918
"(CASSVGVNTEAFF, TCRBV09, TCRBV09-01, null, TCRBJ01, TCRBJ01-01, 01)",7.692505e-06,0.000000,0.006403


In [54]:
tcr = p_value_df[p_value_df.p_value<=0.02].T.columns.values

In [5]:
from sklearn.ensemble import RandomForestClassifier

In [6]:
clf = RandomForestClassifier(bootstrap=True, class_weight= None, criterion='gini',
                                 max_depth=10, max_features='auto', max_leaf_nodes=None,
                                 min_impurity_decrease=0.0, min_impurity_split=None,
                                 min_samples_leaf=1, min_samples_split=2,
                                 min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=-1,
                                 oob_score=True, random_state=0, verbose=0, warm_start=False)
    # num_comp = 100

In [63]:
tcr = TCRs_selection_ttest(count_train,TCRs,0.02)

In [74]:
sample_name = []
y_test = []
    
pred = []
pred_prob = []

kf = LeaveOneOut()
for train_index,test_index in kf.split(freq_df):
    train_cv, test_cv = freq_df.iloc[train_index], freq_df.iloc[test_index]

    sample_name.append(test_cv['sample_name'].tolist()[0])
    y_test.append(test_cv['phenotype_status'].values[0])
    train = train_cv.drop(['sample_name','phenotype_status','total_TCRs'],axis=1)
    test = test_cv.drop(['sample_name','phenotype_status','total_TCRs'],axis=1)

    asso_TCRs = TCRs_selection_ttest(train_cv,TCRs,0.02)
    print('Length of phenotype_associated TCRs:',len(asso_TCRs))
    clf.fit(train[asso_TCRs],train_cv['phenotype_status'])

    y_pred = clf.predict(test[asso_TCRs])[0]
    pred.append(y_pred)
    y_prob = clf.predict_proba(test[asso_TCRs])[:,1][0]
    pred_prob.append(y_prob)

    print(test_cv['sample_name'].tolist()[0])
    print('y_true:',test_cv['phenotype_status'].values[0],'y_pred:',y_pred,'posterior_c1:',y_prob)
    print()

print('cv auroc:',roc_auc_score(y_test,pred_prob))
print('cv log_loss:',log_loss(y_test,pred_prob))

Length of phenotype_associated TCRs: 85
RA47
y_true: 1 y_pred: 0 posterior_c1: 0.447973755369751

Length of phenotype_associated TCRs: 93
HC9
y_true: 0 y_pred: 1 posterior_c1: 0.5820826818878616

Length of phenotype_associated TCRs: 83
RA29
y_true: 1 y_pred: 1 posterior_c1: 0.6117146909501732

Length of phenotype_associated TCRs: 92
RA8
y_true: 1 y_pred: 1 posterior_c1: 0.5220011123639112

Length of phenotype_associated TCRs: 92
RA63
y_true: 1 y_pred: 1 posterior_c1: 0.7669194925041782

Length of phenotype_associated TCRs: 89
RA33
y_true: 1 y_pred: 1 posterior_c1: 0.9205225656776033

Length of phenotype_associated TCRs: 98
HC17
y_true: 0 y_pred: 1 posterior_c1: 0.6271068755311642

Length of phenotype_associated TCRs: 95
HC3
y_true: 0 y_pred: 1 posterior_c1: 0.6068503036437359

Length of phenotype_associated TCRs: 95
RA17
y_true: 1 y_pred: 1 posterior_c1: 0.6177849612943507

Length of phenotype_associated TCRs: 83
RA46
y_true: 1 y_pred: 1 posterior_c1: 0.6509455647168726

Length of phen

Length of phenotype_associated TCRs: 87
RA76
y_true: 1 y_pred: 0 posterior_c1: 0.49456914534390933

Length of phenotype_associated TCRs: 99
HC16
y_true: 0 y_pred: 1 posterior_c1: 0.5511451374615413

cv auroc: 0.46307692307692305
cv log_loss: 0.63618375324774


In [75]:
sample_name = []
y_test = []
    
pred = []
pred_prob = []

kf = LeaveOneOut()
for train_index,test_index in kf.split(freq_df):
    train_cv, test_cv = freq_df.iloc[train_index], freq_df.iloc[test_index]

    sample_name.append(test_cv['sample_name'].tolist()[0])
    y_test.append(test_cv['phenotype_status'].values[0])
    train = train_cv.drop(['sample_name','phenotype_status','total_TCRs'],axis=1)
    test = test_cv.drop(['sample_name','phenotype_status','total_TCRs'],axis=1)

    asso_TCRs = TCRs_selection_ttest(train_cv,TCRs,0.04)
    print('Length of phenotype_associated TCRs:',len(asso_TCRs))
    clf.fit(train[asso_TCRs],train_cv['phenotype_status'])

    y_pred = clf.predict(test[asso_TCRs])[0]
    pred.append(y_pred)
    y_prob = clf.predict_proba(test[asso_TCRs])[:,1][0]
    pred_prob.append(y_prob)

    print(test_cv['sample_name'].tolist()[0])
    print('y_true:',test_cv['phenotype_status'].values[0],'y_pred:',y_pred,'posterior_c1:',y_prob)
    print()

print('cv auroc:',roc_auc_score(y_test,pred_prob))
print('cv log_loss:',log_loss(y_test,pred_prob))

Length of phenotype_associated TCRs: 354
RA47
y_true: 1 y_pred: 1 posterior_c1: 0.5258541689620668

Length of phenotype_associated TCRs: 372
HC9
y_true: 0 y_pred: 1 posterior_c1: 0.5310426349710403

Length of phenotype_associated TCRs: 344
RA29
y_true: 1 y_pred: 1 posterior_c1: 0.6682171853208773

Length of phenotype_associated TCRs: 368
RA8
y_true: 1 y_pred: 1 posterior_c1: 0.5394322670429572

Length of phenotype_associated TCRs: 368
RA63
y_true: 1 y_pred: 1 posterior_c1: 0.5736220926453418

Length of phenotype_associated TCRs: 350
RA33
y_true: 1 y_pred: 1 posterior_c1: 0.8522014682719141

Length of phenotype_associated TCRs: 390
HC17
y_true: 0 y_pred: 1 posterior_c1: 0.5950904337922456

Length of phenotype_associated TCRs: 376
HC3
y_true: 0 y_pred: 1 posterior_c1: 0.5438467847100821

Length of phenotype_associated TCRs: 371
RA17
y_true: 1 y_pred: 0 posterior_c1: 0.4843003212876058

Length of phenotype_associated TCRs: 344
RA46
y_true: 1 y_pred: 1 posterior_c1: 0.6908574531647517

Len

RA75
y_true: 1 y_pred: 1 posterior_c1: 0.8260549391180884

Length of phenotype_associated TCRs: 350
RA76
y_true: 1 y_pred: 1 posterior_c1: 0.5572873806500802

Length of phenotype_associated TCRs: 386
HC16
y_true: 0 y_pred: 1 posterior_c1: 0.6128570890744541

cv auroc: 0.3769230769230769
cv log_loss: 0.6803482283896946


In [16]:
sample_name = []
y_test = []
    
pred = []
pred_prob = []

kf = LeaveOneOut()
for train_index,test_index in kf.split(freq_df):
    train_cv, test_cv = freq_df.iloc[train_index], freq_df.iloc[test_index]

    sample_name.append(test_cv['sample_name'].tolist()[0])
    y_test.append(test_cv['phenotype_status'].values[0])
    train = train_cv.drop(['sample_name','phenotype_status','total_TCRs'],axis=1)
    test = test_cv.drop(['sample_name','phenotype_status','total_TCRs'],axis=1)

    asso_TCRs = TCRs_selection_ttest(train_cv,TCRs,0.03)
    print('Length of phenotype_associated TCRs:',len(asso_TCRs))
    clf.fit(train[asso_TCRs],train_cv['phenotype_status'])

    y_pred = clf.predict(test[asso_TCRs])[0]
    pred.append(y_pred)
    y_prob = clf.predict_proba(test[asso_TCRs])[:,1][0]
    pred_prob.append(y_prob)

    print(test_cv['sample_name'].tolist()[0])
    print('y_true:',test_cv['phenotype_status'].values[0],'y_pred:',y_pred,'posterior_c1:',y_prob)
    print()

print('cv auroc:',roc_auc_score(y_test,pred_prob))
print('cv log_loss:',log_loss(y_test,pred_prob))

Length of phenotype_associated TCRs: 244
RA47
y_true: 1 y_pred: 1 posterior_c1: 0.5471088932745309

Length of phenotype_associated TCRs: 259
HC9
y_true: 0 y_pred: 1 posterior_c1: 0.561332836994919

Length of phenotype_associated TCRs: 234
RA29
y_true: 1 y_pred: 1 posterior_c1: 0.5451415447598976

Length of phenotype_associated TCRs: 257
RA8
y_true: 1 y_pred: 1 posterior_c1: 0.5839200119788598

Length of phenotype_associated TCRs: 258
RA63
y_true: 1 y_pred: 1 posterior_c1: 0.6624800809013842

Length of phenotype_associated TCRs: 250
RA33
y_true: 1 y_pred: 1 posterior_c1: 0.9140513085700509

Length of phenotype_associated TCRs: 277
HC17
y_true: 0 y_pred: 1 posterior_c1: 0.5799629243368392

Length of phenotype_associated TCRs: 263
HC3
y_true: 0 y_pred: 1 posterior_c1: 0.5858476894248829

Length of phenotype_associated TCRs: 259
RA17
y_true: 1 y_pred: 1 posterior_c1: 0.5597152998309444

Length of phenotype_associated TCRs: 238
RA46
y_true: 1 y_pred: 1 posterior_c1: 0.6980802070957752

Leng

RA75
y_true: 1 y_pred: 1 posterior_c1: 0.8990343887138796

Length of phenotype_associated TCRs: 241
RA76
y_true: 1 y_pred: 1 posterior_c1: 0.5236235814308318

Length of phenotype_associated TCRs: 266
HC16
y_true: 0 y_pred: 1 posterior_c1: 0.5185881980393539

cv auroc: 0.4107692307692307
cv log_loss: 0.6545977138679123


In [26]:
def t_test(pos,neg):

    t_stat, pvalue = ttest_ind(pos,neg,equal_var=False)
    if t_stat > 0:
        pvalue = pvalue/2
    else:
        pvalue = 1-(pvalue/2)
    
    return pvalue


def TCRs_selection_ttest(train,kwargs):
    '''
    Select TCRs based on Fisher exact test p-values
    Args:
        count_train: pandas dataframe, count version of training set
        TCRs: a list of TCR candidates
        threshold: float(>0), p-value threshold 
    Return 
        a list of selected TCRs
    '''
    TCRs = train.drop(['sample_name','phenotype_status','total_TCRs'],axis=1).columns.values # a list of TCR candidates

    # the subdf of different phenotype_status(negative/positive)
    neg_df = train[train['phenotype_status']==0]
    pos_df = train[train['phenotype_status']==1]

    '''
    Construct a TCR incidence dictionary
    key is TCR, value is [num_present_c0,num_present_c1]
    '''
    TCRs_pvalue = dict() # init dict
    for tcr in TCRs: # for each TCR in the list of TCR candidates
        
        pos = pos_df[tcr]
        neg = neg_df[tcr]

        if np.mean(pos)!=0 or np.mean(neg)!=0: # if does not occur in any classes, not to add to the dict 
            TCRs_pvalue[tcr] = [np.mean(pos),np.mean(neg)] # add the TCR incidence to dict
            p_value = t_test(pos,neg)
            TCRs_pvalue[tcr].append(p_value) # store p-value

    p_value_df = pd.DataFrame.from_dict(TCRs_pvalue,orient='index') # generate df from TCR incidence dict
    p_value_df.columns = ['mean(pos)','mean(neg)','p_value'] # rename df

    threshold = kwargs['threshold']
    TCRs_asso = p_value_df[p_value_df.p_value<=threshold].T.columns.values # select a list of pehnotype_associated TCRs
    return TCRs_asso


In [16]:
def LOOCV(data,clf,feature_selection,verbose=True,whether_freq=False,**kwargs):
    '''
    Perform Leave-One-Out cross validation
    Args:
        train_origin: training dataframe, containing columns 'sample_name','phenotype_status' and 'unique_TCRs'
        count_df: count version dataframe
        threshold: p-value threshold
        priors_init_value: [[a_c0,b_c0],[a_c1,b_c1]], beta prior initialization values
        prior_init_fun: the function used to initialize priors
        verbose: bool value, whether output more information
    '''
    if whether_freq == False:
        TCRs = data.drop(['sample_name','phenotype_status'],axis=1).columns.values # a list of TCR candidates
    else:
        TCRs = data.drop(['sample_name','phenotype_status','total_TCRs'],axis=1).columns.values # a list of TCR candidates
    # Init lists
    sample_name = []
    y_true = []
    y_pred = []
    y_proba = [] 

    kf = LeaveOneOut()
    for train_index,test_index in kf.split(data):
        train_cv, test_cv = data.iloc[train_index], data.iloc[test_index]

        train = train_cv.drop(['sample_name','phenotype_status'],axis=1)
        test = test_cv.drop(['sample_name','phenotype_status'],axis=1)

        asso_TCRs = feature_selection(train_cv,kwargs)
        print('Length of phenotype_associated TCRs:',len(asso_TCRs))

        clf.fit(train[asso_TCRs],train_cv['phenotype_status'])
        test_pred = clf.predict(test[asso_TCRs])[0]
        test_prob = clf.predict_proba(test[asso_TCRs])[:,1][0]

        sample_name.append(test_cv['sample_name'].tolist()[0])
        y_true.append(test_cv['phenotype_status'].values[0])
        y_pred.append(test_pred)
        y_proba.append(test_prob)

        print(test_cv['sample_name'].tolist()[0])
        print('y_true:',test_cv['phenotype_status'].values[0],'y_pred:',test_pred,'posterior_c1:',test_prob)
        print()

    print('cv auroc:',roc_auc_score(y_true,y_proba))
    print('cv log_loss:',log_loss(y_true,y_proba))


#     prediction = pd.DataFrame({'sample_name':sample_name,'y_true:':y_test,'pred':pred,'posterior_c1':pred_prob})
#     now = datetime.now()
#     currentDate = str(now.month) + "_" + str(now.day) + "_" + str(now.year)+"_"+str(now.hour)+"_"+str(now.minute)+"_"+str(now.second)
#     prediction.to_csv('results/RF-pred_'+currentDate+'.csv',index=False)




In [27]:
LOOCV(freq_df,clf,TCRs_selection_ttest,**{'threshold':0.02})

Length of phenotype_associated TCRs: 85
RA47
y_true: 1 y_pred: 0 posterior_c1: 0.447973755369751

Length of phenotype_associated TCRs: 93
HC9
y_true: 0 y_pred: 1 posterior_c1: 0.5820826818878616

Length of phenotype_associated TCRs: 83
RA29
y_true: 1 y_pred: 1 posterior_c1: 0.6117146909501732

Length of phenotype_associated TCRs: 92
RA8
y_true: 1 y_pred: 1 posterior_c1: 0.5220011123639112

Length of phenotype_associated TCRs: 92
RA63
y_true: 1 y_pred: 1 posterior_c1: 0.7669194925041782

Length of phenotype_associated TCRs: 89
RA33
y_true: 1 y_pred: 1 posterior_c1: 0.9205225656776033

Length of phenotype_associated TCRs: 98
HC17
y_true: 0 y_pred: 1 posterior_c1: 0.6271068755311642

Length of phenotype_associated TCRs: 95
HC3
y_true: 0 y_pred: 1 posterior_c1: 0.6068503036437359

Length of phenotype_associated TCRs: 95
RA17
y_true: 1 y_pred: 1 posterior_c1: 0.6177849612943507

Length of phenotype_associated TCRs: 83
RA46
y_true: 1 y_pred: 1 posterior_c1: 0.6509455647168726

Length of phen

Length of phenotype_associated TCRs: 87
RA76
y_true: 1 y_pred: 0 posterior_c1: 0.49456914534390933

Length of phenotype_associated TCRs: 99
HC16
y_true: 0 y_pred: 1 posterior_c1: 0.5511451374615413

cv auroc: 0.46307692307692305
cv log_loss: 0.63618375324774
